In [2]:
import gym

env = gym.make("Taxi-v3", render_mode="ansi")

env.reset() # reset environment to a new, random state


print("Action Space {}".format(env.action_space))
print("State Space {}".format(env.observation_space))

Action Space Discrete(6)
State Space Discrete(500)


In [3]:
state = env.encode(3, 3, 2, 0) # (taxi row, taxi column, passenger index, destination index)
print("State:", state)

env.s = state
env.render()


State: 368


'+---------+\n|R: |\x1b \x1b: :G|\n| : | : : |\n| : : : : |\n| | : | : |\n|\x1bY\x1b| : |\x1bB\x1b: |\n+---------+\n\n'

In [4]:
import numpy as np
q_table = np.zeros([env.observation_space.n, env.action_space.n])

In [12]:
print(q_table.shape)
print(np.argmax(q_table[300]) )

(500, 6)
3


In [6]:
%%time
"""Training the agent"""

import random
from IPython.display import clear_output

# Hyperparameters
alpha = 0.1
gamma = 0.6
epsilon = 0.1

# For plotting metrics
all_epochs = []
all_penalties = []

for i in range(1, 100):
    state = env.reset()[0]

    epochs = 0
    penalties = 0
    reward = 0
    done = False
    
    while not done:
        #env.render()
        if random.uniform(0, 1) < epsilon:
            action = env.action_space.sample() # Explore action space
        else:

            action = np.argmax(q_table[state]) # Exploit learned values

        next_state, reward, done, done, info = env.step(action) 

        
        old_value = q_table[state, action]
        next_max = np.max(q_table[next_state])
        
        new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_max)
        q_table[state, action] = new_value

        if reward == -10:
            penalties += 1

        state = next_state
        epochs += 1
    
    if i % 100 == 0:

        clear_output(wait=True)
        print(f"Episode: {i}")
    
print("Training finished.\n")

/Users/bastianchuttarsing/Documents/ReinforcmentLearning_project1/.venv/lib/python3.9/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


Training finished.

CPU times: user 770 ms, sys: 116 ms, total: 885 ms
Wall time: 262 ms


In [8]:
"""Evaluate agent's performance after Q-learning"""
#env = gym.make("Taxi-v3", render_mode="human")

total_epochs, total_penalties = 0, 0
episodes = 100

for _ in range(episodes):
    state = env.reset()[0]
    epochs, penalties, reward = 0, 0, 0
    
    done = False
    
    while not done:
        action = np.argmax(q_table[state])
        #action = env.action_space.sample() # random action
        state, reward, done, done, info = env.step(action)

        if reward == -10:
            penalties += 1

        epochs += 1

    total_penalties += penalties
    total_epochs += epochs
    env.reset()

print(f"Results after {episodes} episodes:")
print(f"Average timesteps per episode: {total_epochs / episodes}")
print(f"Average penalties per episode: {total_penalties / episodes}")

Results after 100 episodes:
Average timesteps per episode: 200.0
Average penalties per episode: 5.94


In [11]:
q_table[0]

array([-0.19      , -0.1       , -0.30968009,  1.08547407, 16.78865767,
       -0.30104217])